In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from utils import count_parameters, jokes_collate_fn

import pandas as pd

import torch
from torch.utils.tensorboard import SummaryWriter
from torch.optim import AdamW

from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Model, GPT2LMHeadModel, GPT2Tokenizer, pipeline

# Model loading

In [3]:
model_id = "gpt2"
device = "cuda:1"

device_map = {
    1: [0, 1, 2],
    2: [3, 4, 5],
    3: [6, 7, 8],
    4: [9, 10, 11]
}

tokenizer = GPT2Tokenizer.from_pretrained(model_id)
model = GPT2LMHeadModel.from_pretrained(model_id)

In [4]:
count_parameters(model)

124.44M parameters


In [5]:
model.parallelize(device_map)

In [6]:
special_tokens = {
    'pad_token': '<|padtoken|>',
    'unk_token': '<|unknowntoken|>'
}
tokenizer.add_special_tokens(special_tokens)
model.resize_token_embeddings(len(tokenizer))

Embedding(50258, 768)

# Dataset

## ShortJokes

Dataset link: https://www.kaggle.com/abhinavmoudgil95/short-jokes

### Train test split

In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
data = pd.read_csv("data/shortjokes/shortjokes.csv")

In [ ]:
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)
val_data, test_data = train_test_split(val_data, test_size=0.5, random_state=42)

In [ ]:
train_data.to_csv("data/shortjokes/train.csv", index=False)
val_data.to_csv("data/shortjokes/val.csv", index=False)
test_data.to_csv("data/shortjokes/test.csv", index=False)

In [ ]:
data.shape

In [ ]:
tokenizer_lengths = data["Joke"].map(lambda x: len(tokenizer.encode_plus(x)['input_ids']))
plt.hist(tokenizer_lengths, bins=40)
plt.show()

In [ ]:
tokenizer_lengths.max()

In [ ]:
tokenizer_lengths.min()

### Preparing data

In [7]:
class JokesDataset(Dataset):
    def __init__(self, jokes_filepath='data/shortjokes.csv'):
        data = pd.read_csv(jokes_filepath)
        self.jokes = data["Joke"].values
        
    def __len__(self):
        return len(self.jokes)
    
    def __getitem__(self, item):
        return self.jokes[item]

In [8]:
train_data = JokesDataset(jokes_filepath='data/shortjokes/train.csv')
test_data = JokesDataset(jokes_filepath='data/shortjokes/test.csv')
val_data = JokesDataset(jokes_filepath='data/shortjokes/val.csv')

In [9]:
def jokes_collate_fn(batch):
    batch_encoded = tokenizer.batch_encode_plus(
        batch,
        padding='max_length',
        return_tensors='pt',
        max_length=128
    )
    input_ids = batch_encoded['input_ids'].to(device)
    attention_mask = batch_encoded['attention_mask'].to(device)
    labels = input_ids.masked_fill(input_ids == tokenizer.pad_token_id, -100)
    return (input_ids, attention_mask, labels)

In [10]:
train_dataloader = DataLoader(train_data, batch_size=3, shuffle=True, collate_fn=jokes_collate_fn)
val_dataloader = DataLoader(val_data, batch_size=3, collate_fn=jokes_collate_fn)

## Training the model

In [11]:
from tqdm.notebook import tqdm
from utils import train_model_with_toploss

In [12]:
# %load_ext tensorboard
# %tensorboard --logdir logs/ --port 8800

In [13]:
directory = "logs/jokes/with_toploss"
model = train_model_with_toploss(
    model, tokenizer, 
    train_dataloader, 
    val_dataloader, 
    epochs=1,
    logging=directory, 
    use_labels=False
)
model.save_pretrained(directory + "/model_1epoch")

Training on epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Calculating topological loss:   0%|          | 0/20 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Calculating topological loss:   0%|          | 0/20 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Calculating topological loss:   0%|          | 0/20 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Calculating topological loss:   0%|          | 0/20 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

In [ ]:
# model.save_pretrained(directory + "/model_1epoch")

## Perplexity calculation

In [3]:
test_texts = pd.read_csv("data/shortjokes/test.csv")

In [4]:
test_texts.shape

(23166, 2)

In [5]:
from utils import calc_ppl

In [10]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
special_tokens = {
    'pad_token': '<|padtoken|>',
    'unk_token': '<|unknowntoken|>'
}
tokenizer.add_special_tokens(special_tokens)

1

In [6]:
model = GPT2LMHeadModel.from_pretrained("logs/jokes/with_toploss/model_1epoch")
model.to("cuda:1")

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50258, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [23]:
calc_ppl(model, text_data=test_texts["Joke"].values, tokenizer=tokenizer, stride=128, device=model.device)

  0%|          | 0/4498 [00:00<?, ?it/s]

262.0511169433594

In [16]:
model_2 = GPT2LMHeadModel.from_pretrained("logs/jokes/default/model_1epoch")
model_2.to("cuda:2")

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50258, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [30]:
calc_ppl(model_2, text_data=test_texts["Joke"].values, tokenizer=tokenizer, stride=128, device=model_2.device)

  0%|          | 0/4498 [00:00<?, ?it/s]

170.43780517578125

In [35]:
import numpy as np
print(f"Perplexity with topological loss: {2**(np.log(262.05))}")
print(f"Perplexity with only standard loss: {2**(np.log(170.43780517578125))}")

Perplexity with topological loss: 47.45655023599167
Perplexity with only standard loss: 35.22115832798323


## Generate examples

In [11]:
from utils import generate_sentences

In [13]:
model.device

device(type='cuda', index=1)

In [14]:
generate_sentences(model, tokenizer, num_return_sequences=5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[' today is a time to take your pants off to get ready for the night.',
 ' this is probably the joke I have all the time right now. Why do blondes always wear bra sizes? to hide. Because they hate looking in the mirror.',
 " in New Zealand What is the difference between New Zealander USA and a cow? Beef can't swim.",
 "This will be my 7th diet to eat food that doesn't belong in our food. My husband has told me this once, so I'm going to eat that!",
 " should have its own set of rules but it's kinda pointless"]

In [19]:
generate_sentences(model_2, tokenizer, num_return_sequences=10)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


["I haven't been able to enjoy my new haircut It has made my head hurt more.",
 'My girlfriend is looking for me so I decided not to follow her in, instead I just keep walking around saying "I love you" and not taking her notice.',
 'Lucky guy, just saw a woman sitting naked staring at a screen. What a smart guy!',
 'Hey baby, why are you wearing a tuxedo? Because your dad is a ghost..',
 "Noah's Ark is so much fun the world over. I thought it was over... But then I realized it's not.",
 "No, but I'll put my tongue down you're gonna have a little dick about a little fish too",
 "Owesome, not really a sexual experience. But if you've ever been in love, you know you're getting pretty fucking laid if you bring two guys together.",
 "Walking into a grocery shop.... There's a Target employee named Amy wearing a Target Target t-shirt and looking like a Target employee. That's not you. You are what I want you to be.",
 "Do you mind if I join you on a date? Yeah, I know. But the date's great, 

## Distributions plot

In [28]:
import imageio, os
from collections import defaultdict

In [26]:
filenames = os.listdir("logs/jokes/with_toploss/pictures")

In [42]:
sorted_filenames = defaultdict(dict)

In [43]:
for filename in filenames:
    n = int(filename.split("_")[0][1:])
    l = int(filename.split("_")[1][1:])
    h = int(filename.split("_")[2].split('.png')[0][1:])
    if h in sorted_filenames[l]:
        sorted_filenames[l][h].append(filename)
    else:
        sorted_filenames[l][h] = [filename]

In [48]:
from tqdm.auto import tqdm

In [54]:
images = []
for layer in tqdm(range(12), desc="Getting gifs"):
    for head in range(12):
        images = [
            imageio.imread("logs/jokes/with_toploss/pictures/" + filename) for filename in sorted(sorted_filenames[layer][head], key=lambda x: int(x.split("_")[0][1:]))
        ]
        imageio.mimsave(f'logs/jokes/with_toploss/gifs/l{layer}_h{head}.gif', images, duration=1)

Getting gifs:   0%|          | 0/12 [00:00<?, ?it/s]

## SLOR